<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Minimum%20Spanning%20Tree%20(Animation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimum Spanning Tree (Animation)

## Implementierung

In [17]:
%matplotlib notebook

import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import networkx as nx
import numpy as np
import pandas as pd

from IPython.display import HTML
from matplotlib.widgets import Button

In [19]:
# Set random seed for reproducibility
np.random.seed(1)

# Create a random points (=nodes)
number_of_points = 30
points = list(range(number_of_points))
nodes = [(i, {'x': np.random.randint(1, 99), 'y': np.random.randint(1, 99)}) for i in points]

# Create graph
G = nx.Graph()
G.add_nodes_from(nodes)

# Calculate distances and create edges
for o in points:
    for d in points:
        distance_o_to_d = np.linalg.norm([G.nodes[o]['x'] - G.nodes[d]['x'], G.nodes[o]['y'] - G.nodes[d]['y']])
        G.add_edge(o, d, weight=round(distance_o_to_d, 2))

# Calculate minimum spanning tree (in an undirected graph)
G.to_undirected()
T = nx.minimum_spanning_tree(G=G)

# Get MST edges
MST_edges = T.edges(data=True)
sorted_MST_edges = sorted(MST_edges, key=lambda x: x[2]['weight'])

# Initialize data frames for points and lines
line_data = {'o_x': [], 'o_y': [], 'd_x': [], 'd_y': [], 'iteration': []}
point_data = {'x': [], 'y': [], 'iteration': [], 'level': [], 'radii': []}

# Initialize node levels
node_level = {p: 0 for p in points}

# Initialize a subgraph (capturing the MST per iteration)
subG = nx.Graph()
subG.add_nodes_from(nodes)

# Process MST edges
for iteration, edge in enumerate(sorted_MST_edges):
    line_data['o_x'].append(G.nodes[edge[0]]['x'])
    line_data['o_y'].append(G.nodes[edge[0]]['y'])
    line_data['d_x'].append(G.nodes[edge[1]]['x'])
    line_data['d_y'].append(G.nodes[edge[1]]['y'])
    line_data['iteration'].append(iteration)

    radii = round(edge[2]['weight'] / 2, 4)

    for p in points:
        point_data['x'].append(G.nodes[p]['x'])
        point_data['y'].append(G.nodes[p]['y'])
        point_data['iteration'].append(iteration)
        point_data['level'].append(node_level[p])
        point_data['radii'].append(radii)

    subG.add_edges_from([edge])

    # Increase node levels for origin's descendants
    descendants = nx.algorithms.descendants(subG, edge[0])
    max_level = max(node_level[d] for d in descendants)
    node_level[edge[0]] = max_level + 1
    for d in descendants:
        node_level[d] = max_level + 1

# Create data frames
df_lines = pd.DataFrame(line_data)
df_points = pd.DataFrame(point_data)

# Create initial plot
fig, ax = plt.subplots()
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_aspect('equal', adjustable='box')

# Generate random colors for points
colors = [np.random.rand(3,) for _ in range(len(points))]

# Plotting functions
def plot_shapes(frame):
    ax.clear()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_aspect('equal', adjustable='box')

    df_points_selected = df_points[df_points['iteration'] <= frame]
    df_points_selected = df_points_selected.sort_values(by='level', ascending=False)

    for index, row in df_points_selected.iterrows():
        circle_obj = plt.Circle((row['x'], row['y']), row['radii'], fill=True, color=colors[int(row['level'])])
        ax.add_patch(circle_obj)

    for p in points:
        circle_obj = plt.Circle((G.nodes[p]['x'], G.nodes[p]['y']), 0.3, fill=True, color='black')
        ax.add_patch(circle_obj)

    df_lines_selected = df_lines[df_lines['iteration'] <= frame]
    for index, row in df_lines_selected.iterrows():
        ax.plot([row['o_x'], row['d_x']], [row['o_y'], row['d_y']], label='Line', color='black')

    ax.set_title(f"Iteration: {frame}")

# Animation update function
def update(frame):
    plot_shapes(frame)

# Create animation
ani = animation.FuncAnimation(fig, update, frames=number_of_points, interval=500, repeat=True)

# Show animation
HTML(ani.to_jshtml())

<IPython.core.display.Javascript object>